# Approximate solvers for the Euler equations of gas dynamics

To do:
- More pedagogy?
- Implement entropy fix for Roe solver (and discuss it?)
- Discuss positivity of HLL solver
- Fix plotting of multiple solutions so nothing is clipped
- Decide what phase plane plots to include and add them

In [Part I](Euler_equations.ipynb) we studied the Riemann problem for Euler equations of inviscid, compressible fluid flow .  As we saw, the exact solution of the Riemann problem is computationally expensive, since it requires solving a set of nonlinear algebraic equations.  In the context of finite volume methods, the detailed structure of the Riemann solution is almost immediately discarded -- only its impact on the neighboring cell averages is used.  So it makes sense to consider whether we can approximate the solution with less computation.  In this chapter, we investigate approximate solvers for the Euler equations.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize

import numpy as np
from exact_solvers import Euler
from clawpack import riemann
from utils import riemann_tools
import matplotlib.pyplot as plt
from collections import namedtuple
from ipywidgets import interact
from ipywidgets import widgets
import matplotlib
Primitive_State = namedtuple('State', Euler.primitive_variables)
gamma = 1.4
problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0

## Shock tube problem

We start with the same simple shock tube problem from [the Euler equations chapter](Euler_equations.ipynb) and recall the structure of its exact solution.

In [ ]:
def riemann_solution(left_state, right_state):
    q_left  = Euler.primitive_to_conservative(*left_state)
    q_right = Euler.primitive_to_conservative(*right_state)

    ex_states, ex_speeds, reval, wave_types = Euler.exact_riemann_solution(q_left, q_right, gamma)
    
    plot_function = riemann_tools.make_plot_function(ex_states, ex_speeds, reval, wave_types,
                                                     layout='vertical', 
                                                     variable_names=Euler.primitive_variables,
                                                     plot_chars=[Euler.lambda1,Euler.lambda2,Euler.lambda3],
                                                     derived_variables=Euler.cons_to_prim)

    interact(plot_function, t=widgets.FloatSlider(value=0.1,min=0,max=.9),
             which_char=widgets.Dropdown(options=[None,1,2,3],description='Show characteristics'))

In [ ]:
from ipywidgets import interact                   # for interactive widgets
#from utils.snapshot_widgets import interact      # for static figure that can be viewed online

In [ ]:
left_state  = Primitive_State(Density = 3.,
                              Velocity = 0.,
                              Pressure = 3.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 0.,
                              Pressure = 1.)

q_left  = Euler.primitive_to_conservative(*left_state)
q_right = Euler.primitive_to_conservative(*right_state)

riemann_solution(left_state,right_state)

## HLLE Solver

The HLLE solver uses only two waves with a constant state between that is uniquely defined by conservation for any choice of the two wave speeds.  The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characteristic speed $\lambda^3$ in the right state $q_r$.

In [ ]:
hll_solver = riemann.euler_1D_py.euler_hll_1D

states_hll, s_hll, hll_eval = riemann_tools.riemann_solution(hll_solver,
                                            q_left, q_right,
                                            problem_data=problem_data)

In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(q_left ,q_right, gamma)

plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll],
                                     [ex_speeds,s_hll],
                                     [reval,hll_eval],
                                     [ex_wave_types,['contact']*2],
                                     ['Exact','HLLE'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

#### Phase plane plots

Need to improve the phase plane plots for Euler equations with 3 variables.  Below is a first attempt.  The axis labels are wrong; these are actually primitive coordinates.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=figsize)
riemann_tools.plot_phase(states_hll,0,1,ax[0])
riemann_tools.plot_phase(states_hll,0,2,ax[1])
riemann_tools.plot_phase(states_hll,1,2,ax[2])
riemann_tools.plot_phase_3d(states_hll)

## Roe solver
The Roe solver is an example of a linearized Riemann solver.  It approximates the Riemann problem by considering an approximation of the flux Jacobian: $\hat{A} \approx f'(q)$ and exactly solving the Riemann problem for the linear hyperbolic system

$$q_t + \hat{A}q_x = 0.$$

The Roe linearization is chosen so that in the case that $(q_l, q_r)$ are related by a single shock, the Roe solver gives the exact solution.

In [ ]:
roe_solver = riemann.euler_1D_py.euler_roe_1D
num_eqn = riemann.euler_1D_py.num_eqn

problem_data['efix'] = False

states_roe, s_roe, eval_roe = riemann_tools.riemann_solution(roe_solver,
                                    q_left,q_right,
                                    problem_data=problem_data)

#### Phase plane plots

In [ ]:
fig, ax = plt.subplots(1,2,figsize=figsize)
riemann_tools.plot_phase(states_roe,0,1,ax[0])
riemann_tools.plot_phase(states_roe,0,2,ax[1])
riemann_tools.plot_phase_3d(states_roe)

In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(q_left ,q_right, gamma)

plot_function = \
    riemann_tools.make_plot_function([ex_states, states_roe],
                                     [ex_speeds, s_roe],
                                     [reval, eval_roe],
                                     [ex_wave_types,['contact']*3],
                                     ['Exact','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## Comparison of two approximate solvers with the exact solution


In [ ]:
plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll,states_roe],
                                     [ex_speeds,s_hll,s_roe],
                                     [reval,hll_eval,eval_roe],
                                     [ex_wave_types,['contact']*2,['contact']*3],
                                     ['Exact','HLLE','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## Failure of the Roe solver
For problems with near-vacuum states or strong expansions, any linearized solver may lead to unphysical states.  This happens for the Roe solver in the symmetric expansion problem considered in [the Euler equations chapter](Euler_equations.ipynb).  Here we revisit the problem and see that the Roe solver predicts a negative density value. (what about the entropy fix here?)

(add phase plane plots -- something like Figure 15.3 of the red book)

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = -3.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = 3.,
                              Pressure = 1.)
q_left  = Euler.primitive_to_conservative(*left_state)
q_right = Euler.primitive_to_conservative(*right_state)

In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(q_left, q_right, gamma)

states_hll, s_hll, eval_hll = riemann_tools.riemann_solution(hll_solver,
                                            q_left,q_right,
                                            problem_data=problem_data)

states_roe, s_roe, eval_roe = riemann_tools.riemann_solution(roe_solver,
                                    q_left,q_right,
                                    problem_data=problem_data,verbose=False)

plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll,states_roe],
                                     [ex_speeds,s_hll,s_roe],
                                     [reval,eval_hll,eval_roe],
                                     [ex_wave_types,['contact']*2,['contact']*3],
                                     ['Exact','HLLE','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## Colliding flows
For this problem, the HLL solver picks the Roe wave speeds and there is no contact wave.  As a result, the HLL solution coincides exactly with the Roe solution.

In [ ]:
left_state  = Primitive_State(Density = 1.,
                              Velocity = 3.,
                              Pressure = 1.)
right_state = Primitive_State(Density = 1.,
                              Velocity = -3.,
                              Pressure = 1.)
q_left  = Euler.primitive_to_conservative(*left_state)
q_right = Euler.primitive_to_conservative(*right_state)

In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(q_left ,q_right, gamma)

states_hll, s_hll, eval_hll = riemann_tools.riemann_solution(hll_solver,
                                            q_left,q_right,
                                            problem_data=problem_data)

states_roe, s_roe, eval_roe = riemann_tools.riemann_solution(roe_solver,
                                    q_left,q_right,
                                    problem_data=problem_data,verbose=False)

plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll,states_roe],
                                     [ex_speeds,s_hll,s_roe],
                                     [reval,eval_hll,eval_roe],
                                     [ex_wave_types,['contact']*2,['contact']*3],
                                     ['Exact','HLLE','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## Vacuum states
One of the approximate solvers fails if a density or pressure is set exactly to zero, but both Roe and HLL seem to do fine with near-vacuum initial states.  Of course, they both completely miss the very large velocities generated in the expansion fan.  Notice that HLLE substantially underestimates the 1-wave speed here; is it correct?

In [ ]:
left_state  = Primitive_State(Density = 0.0001,
                              Velocity = 0.,
                              Pressure = 0.0001)
right_state = Primitive_State(Density = 1.,
                              Velocity = -3.,
                              Pressure = 1.)
q_left  = Euler.primitive_to_conservative(*left_state)
q_right = Euler.primitive_to_conservative(*right_state)

In [ ]:
ex_states, ex_speeds, reval, ex_wave_types = \
    Euler.exact_riemann_solution(q_left ,q_right, gamma)

states_hll, s_hll, eval_hll = riemann_tools.riemann_solution(hll_solver,
                                            q_left,q_right,
                                            problem_data=problem_data)

states_roe, s_roe, eval_roe = riemann_tools.riemann_solution(roe_solver,
                                    q_left,q_right,
                                    problem_data=problem_data,verbose=False)

plot_function = \
    riemann_tools.make_plot_function([ex_states,states_hll,states_roe],
                                     [ex_speeds,s_hll,s_roe],
                                     [reval,eval_hll,eval_roe],
                                     [ex_wave_types,['contact']*2,['contact']*3],
                                     ['Exact','HLLE','Roe'],
                                     layout='vertical',
                                     variable_names=Euler.primitive_variables,
                                     derived_variables=Euler.cons_to_prim)
    
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));